In [8]:
import numpy as np
import pandas as pd

In [9]:
dataset = pd.read_excel('dataset.xlsx', sheet_name='AUO')
dataset.head()

,GDS Vd=10V [dark],Vgate,ABSID,GDS Vd=10V [1.5w],Vgate.1,ABSID.1,GDS Vd=10V [2.5w],Vgate.2,ABSID.2,GDS Vd=10V [4w],Vgate.3,ABSID.3
0,NaN,-25.0,2.049800e-11,NaN,-25.0,2.063900e-11,NaN,-25.0,2.248300e-11,NaN,-25.0,2.350100e-11
1,NaN,-24.8,1.216900e-11,NaN,-24.8,1.295900e-11,NaN,-24.8,1.429800e-11,NaN,-24.8,1.522700e-11
2,NaN,-24.6,1.077200e-11,NaN,-24.6,1.155500e-11,NaN,-24.6,1.274500e-11,NaN,-24.6,1.397500e-11
3,NaN,-24.4,9.939000e-12,NaN,-24.4,1.088300e-11,NaN,-24.4,1.216600e-11,NaN,-24.4,1.328200e-11
4,NaN,-24.2,9.705000e-12,NaN,-24.2,1.038500e-11,NaN,-24.2,1.149700e-11,NaN,-24.2,1.258800e-11


In [10]:
row, column = dataset.shape
print("Dataset size : ", row , "x" , column)
print("Null Row : ", row + 1)
print("Null column : ", column + 1)

Dataset size :  502 x 12
Null Row :  503
Null column :  13


In [11]:
colList = dataset.columns.tolist() # All columns
nanCol = [] # Declare null columms
nanIndex = []
for i in range(len(colList)):
    nan = ((dataset[columns_list[i]]).isnull() == True)[0] # nan is True or False
    if(nan): # if nan == true (isnull)
        nanCol.append(colList[i]) 
        nanIndex.append(i)
        
nanIndex.append(column)
print(nanCol)  
print(nanIndex)

NameError: name 'columns_list' is not defined

In [12]:
# Divide into groups 
for i in range(len(nanCol)):
    nanCol[i] = dataset.iloc[0:row, nanIndex[i]:nanIndex[i+1]]

In [13]:
len(nanCol)
nn = nanCol[0].columns[1]
nanCol[0][nn]

IndexError: list index out of range

In [14]:
colName = nanCol[0].columns[1]

value = -10 # search value
newY = [] # new Y List
# https://www.listendata.com/2020/12/how-to-use-variable-in-query-in-pandas.html
# nanCol[3].query("`{0}` == -9".format(colName)) 

if len( nanCol[0].query("`{0}` == @value".format(colName)) ) > 0:
    print('not null')
   
    # show result
    result = dataset.query("`{0}` == @value".format(colName)) 
    print(result)
    
else :
    
    for i in range(len(nanCol)) :
        
        VgateName = nanCol[i].columns[1] # Vgate || Vgate.1 || Vgate.2 ... 
        ABSIDName = nanCol[i].columns[2] # ABSID || ABSID.1 || ABSID.3 ...
        vgateArr = nanCol[i][VgateName].to_numpy()
        arr = dataset[nanCol[3].columns[1]].to_numpy()

        above = vgateArr[np.searchsorted(arr,value,'left')-1]
        below = vgateArr[np.searchsorted(arr,value,'right')]
        print(above, below) 
        X = value
        x1 = min(above, below)
        x2 =  max(above, below)
        dd = nanCol[i].query("`{0}` == @above or `{0}` == @below".format(VgateName))
        dd = dd.sort_index(ascending=True)

        for j in range(2) :
            y1 = dd.query("`{0}` == @x1".format(VgateName))[ABSIDName].to_numpy()[j]
            y2 = dd.query("`{0}` == @x2".format(VgateName))[ABSIDName].to_numpy()[j]
      
        Y = y1 + (X-x1)*(y2-y1)/(x2-x1)
        print(Y)

        newY.extend((0, X, Y))


print(newY)


valList = dataset.columns.tolist()
tf = pd.DataFrame(columns=valList)
tf.loc[len(tf)] = newY
tf


IndexError: list index out of range

In [28]:
# Show close value (include self)
input = -9 # search value
nums = 6 # how many?
d2 = dataset.iloc[(dataset['Vgate']-input).abs().argsort()[:nums]]['Vgate'] # index , value
d2

80    -9.0
421   -9.0
79    -9.2
81    -8.8
422   -9.2
420   -8.8
Name: Vgate, dtype: float64

In [228]:
# not include self value

arr = dataset['Vgate'].to_numpy()
value = -9 # search value
above = arr[np.searchsorted(arr,value,'left')-1]
below = arr[np.searchsorted(arr,value,'right')]
print(above, below) 
X = value
x1 = min(above, below)
x2 =  max(above, below)
dd = dataset.query("Vgate == @above or Vgate == @below")
dd = dd.sort_index(ascending=True)

y1 = dd.query("Vgate == @x1")['ABSID'].to_numpy()[0]
ABSID2_arr = dd.query("Vgate == @x2")['ABSID'].to_numpy()
y2 = ABSID2_arr[0]

print((x1, x2))
print((y1, y2))
print(X)
Y = y1 + (X-x1)*(y2-y1)/(x2-x1)
print(Y)

-9.2 -8.8
(-9.2, -8.8)
(1.45e-13, 2.76e-13)
-9
2.1049999999999999e-13


In [56]:

print("lens : ", len(dd))
dd.iloc[:2]


lens :  4


,GDS Vd=10V [dark],Vgate,ABSID,GDS Vd=10V [1.5w],Vgate.1,ABSID.1,GDS Vd=10V [2.5w],Vgate.2,ABSID.2,GDS Vd=10V [4w],Vgate.3,ABSID.3
79,NaN,-9.2,1.450000e-13,NaN,-9.2,1.660000e-12,NaN,-9.2,2.446000e-12,NaN,-9.2,3.787000e-12
81,NaN,-8.8,2.760000e-13,NaN,-8.8,1.719000e-12,NaN,-8.8,2.628000e-12,NaN,-8.8,3.731000e-12


In [60]:
ABSID_arr = dd['ABSID'].to_numpy()
y1 = ABSID_arr[0]
y2 = ABSID_arr[1]

2.76e-13